In [24]:
import pandas as pd
import os
from tqdm import tqdm
import re

# ==========================================
# --- CẤU HÌNH ĐƯỜNG DẪN ---
# ==========================================
DATA_ROOT = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val"
OUTPUT_ROOT = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data"
RATIOS = ["1-1-1", "2-1-1", "4-1-1", "10-1-1"]

# ==========================================
# --- LOGIC TRÍCH XUẤT CHUẨN MAXENTSCAN ---
# ==========================================

def get_maxent_formats(row):
    seq_full = str(row['sequence']).upper()
    label = int(row['label'])
    
    # Khởi tạo giá trị rỗng (N)
    seq_5 = "N" * 9
    seq_3 = "N" * 23

    mid = len(seq_full) // 2

    # --- TRÍCH XUẤT DONOR (Label 1: Donor, Label 0: Negative) ---
    if label in [0, 1]:
        # Tìm các motif GT
        all_gt = [m.start() for m in re.finditer('GT', seq_full)]
        if all_gt:
            # Ưu tiên GT gần trung tâm nhất
            best_gt = min(all_gt, key=lambda x: abs(x - mid))
            # Cấu trúc: 3bp Exon + GT + 4bp Intron (Tổng 9bp)
            # Vị trí: [GT-3 : GT+6]
            if best_gt >= 3 and (best_gt + 6) <= len(seq_full):
                seq_5 = seq_full[best_gt-3 : best_gt+6]

    # --- TRÍCH XUẤT ACCEPTOR (Label 2: Acceptor, Label 0: Negative) ---
    if label in [0, 2]:
        # Tìm các motif AG
        all_ag = [m.start() for m in re.finditer('AG', seq_full)]
        if all_ag:
            # Ưu tiên AG gần trung tâm nhất
            best_ag = min(all_ag, key=lambda x: abs(x - mid))
            # Cấu trúc: 18bp Intron + AG + 3bp Exon (Tổng 23bp)
            # Vị trí: [AG-18 : AG+5] (Vì AG chiếm 2bp, nên AG+5 lấy thêm 3bp phía sau)
            if best_ag >= 18 and (best_ag + 5) <= len(seq_full):
                seq_3 = seq_full[best_ag-18 : best_ag+5]

    return pd.Series([seq_5, seq_3])

# ==========================================
# --- CHƯƠNG TRÌNH CHÍNH ---
# ==========================================

if __name__ == "__main__":
    os.makedirs(OUTPUT_ROOT, exist_ok=True)
    
    for ratio in RATIOS:
        file_name = f"test_{ratio.replace('-', '_')}.csv"
        input_path = os.path.join(DATA_ROOT, file_name)
        
        if not os.path.exists(input_path):
            print(f"⚠️ Bỏ qua: Không tìm thấy {input_path}")
            continue
            
        print(f"\n🚀 Đang xử lý Ratio: {ratio}")
        df = pd.read_csv(input_path)
        
        # Đồng nhất tên cột label
        if 'Splicing_types' in df.columns:
            df = df.rename(columns={'Splicing_types': 'label'})

        tqdm.pandas(desc=f"Processing {ratio}")
        # Trích xuất 2 cột mới
        df[['seq_5', 'seq_3']] = df.progress_apply(get_maxent_formats, axis=1)
        
        # --- THỐNG KÊ ---
        # Label 1 (Donor) thì seq_5 phải có GT ở vị trí index 3
        donor_check = df[df['label'] == 1]
        valid_5 = donor_check[donor_check['seq_5'].str[3:5] == "GT"].shape[0]
        
        # Label 2 (Acceptor) thì seq_3 phải có AG ở vị trí index 18
        acceptor_check = df[df['label'] == 2]
        valid_3 = acceptor_check[acceptor_check['seq_3'].str[18:20] == "AG"].shape[0]

        print(f"🔍 Thống kê trích xuất:")
        print(f"   - Donor (Label 1): {valid_5}/{len(donor_check)} mẫu hợp lệ.")
        print(f"   - Acceptor (Label 2): {valid_3}/{len(acceptor_check)} mẫu hợp lệ.")

        # --- LƯU FILE ---
        # Lưu file đầy đủ để bạn có thể lọc tùy ý khi chạy MaxEntScan
        output_path = os.path.join(OUTPUT_ROOT, f"maxent_input_{ratio}.csv")
        df.to_csv(output_path, index=False)
        
        # Gợi ý: Tạo file riêng cho Donor và Acceptor để chạy script MaxEnt gốc dễ hơn
        df_donor = df[(df['label'] == 1) & (df['seq_5'].str.len() == 9) & (~df['seq_5'].str.contains('N'))]
        df_acceptor = df[(df['label'] == 2) & (df['seq_3'].str.len() == 23) & (~df['seq_3'].str.contains('N'))]
        
        print(f"✅ Đã lưu file tổng hợp tại: {output_path}")
        print(f"📊 Sẵn sàng: {len(df_donor)} Donor, {len(df_acceptor)} Acceptor.")

    print("\n✨ Hoàn tất!")


🚀 Đang xử lý Ratio: 1-1-1


Processing 1-1-1: 100%|██████████| 26310/26310 [00:03<00:00, 8000.74it/s] 


🔍 Thống kê trích xuất:
   - Donor (Label 1): 8822/8822 mẫu hợp lệ.
   - Acceptor (Label 2): 8666/8666 mẫu hợp lệ.
✅ Đã lưu file tổng hợp tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data\maxent_input_1-1-1.csv
📊 Sẵn sàng: 8822 Donor, 8666 Acceptor.

🚀 Đang xử lý Ratio: 2-1-1


Processing 2-1-1: 100%|██████████| 35132/35132 [00:04<00:00, 7177.19it/s] 


🔍 Thống kê trích xuất:
   - Donor (Label 1): 8822/8822 mẫu hợp lệ.
   - Acceptor (Label 2): 8666/8666 mẫu hợp lệ.
✅ Đã lưu file tổng hợp tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data\maxent_input_2-1-1.csv
📊 Sẵn sàng: 8822 Donor, 8666 Acceptor.

🚀 Đang xử lý Ratio: 4-1-1


Processing 4-1-1: 100%|██████████| 52776/52776 [00:08<00:00, 6479.64it/s] 


🔍 Thống kê trích xuất:
   - Donor (Label 1): 8822/8822 mẫu hợp lệ.
   - Acceptor (Label 2): 8666/8666 mẫu hợp lệ.
✅ Đã lưu file tổng hợp tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data\maxent_input_4-1-1.csv
📊 Sẵn sàng: 8822 Donor, 8666 Acceptor.

🚀 Đang xử lý Ratio: 10-1-1


Processing 10-1-1: 100%|██████████| 105708/105708 [00:18<00:00, 5831.73it/s]


🔍 Thống kê trích xuất:
   - Donor (Label 1): 8822/8822 mẫu hợp lệ.
   - Acceptor (Label 2): 8666/8666 mẫu hợp lệ.
✅ Đã lưu file tổng hợp tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data\maxent_input_10-1-1.csv
📊 Sẵn sàng: 8822 Donor, 8666 Acceptor.

✨ Hoàn tất!


In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm

# Cấu hình đường dẫn
DATA_ROOT = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val"
OUTPUT_ROOT = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

RATIOS = ["1-1-1", "2-1-1", "4-1-1", "10-1-1"]

In [8]:
def extract_smart_motif(row):
    seq_full = str(row['sequence']).upper()
    label = int(row['label'])
    mid = len(seq_full) // 2
    
    seq_5 = "N" * 9
    seq_3 = "N" * 23

    # 1. Xử lý Donor (seq_5): Cần 9bp, GT ở index 3,4
    all_gt = [m.start() for m in re.finditer('GT', seq_full)]
    if all_gt:
        # Tìm GT gần trung tâm chuỗi nhất
        best_gt = min(all_gt, key=lambda x: abs(x - mid))
        if best_gt >= 3 and (best_gt + 6) <= len(seq_full):
            seq_5 = seq_full[best_gt-3 : best_gt+6]

    # 2. Xử lý Acceptor (seq_3): Cần 23bp, AG ở index 18,19
    all_ag = [m.start() for m in re.finditer('AG', seq_full)]
    if all_ag:
        # Tìm AG gần trung tâm chuỗi nhất
        best_ag = min(all_ag, key=lambda x: abs(x - mid))
        if best_ag >= 18 and (best_ag + 5) <= len(seq_full):
            seq_3 = seq_full[best_ag-18 : best_ag+5]
            
    # HẬU KIỂM: Nếu nhãn là 1 mà không có GT đúng chỗ, hoặc nhãn 2 không có AG đúng chỗ
    # thì MaxEntScan sẽ chấm điểm cực thấp, ta để N để dễ theo dõi.
    return pd.Series([seq_5, seq_3])

In [23]:
for ratio in RATIOS:
    file_name = f"test_{ratio.replace('-', '_')}.csv"
    input_path = os.path.join(DATA_ROOT, file_name)
    
    if not os.path.exists(input_path):
        print(f"⚠️ Không tìm thấy: {file_name}")
        continue
        
    df = pd.read_csv(input_path)
    if 'Splicing_types' in df.columns:
        df = df.rename(columns={'Splicing_types': 'label'})

    print(f"\n🚀 Đang xử lý Ratio: {ratio}")
    tqdm.pandas(desc="Extracting")
    df[['seq_5', 'seq_3']] = df.progress_apply(extract_smart_motif, axis=1)
    
    # Kiểm tra tỷ lệ thành công
    d_ok = df[(df['label']==1) & (df['seq_5'].str[3:5] == 'GT')].shape[0]
    a_ok = df[(df['label']==2) & (df['seq_3'].str[18:20] == 'AG')].shape[0]
    
    print(f"📊 Thống kê {ratio}:")
    print(f"   - Donor (Lớp 1) khớp GT: {d_ok}")
    print(f"   - Acceptor (Lớp 2) khớp AG: {a_ok}")

    # Lưu lại (Giữ nguyên số dòng để không làm lệch tập test)
    output_path = os.path.join(OUTPUT_ROOT, f"maxent_{ratio}.csv")
    df.to_csv(output_path, index=False)
    print(f"✅ Đã lưu: {output_path}")


🚀 Đang xử lý Ratio: 1-1-1


Extracting: 100%|██████████| 26310/26310 [00:05<00:00, 4792.61it/s]


📊 Thống kê 1-1-1:
   - Donor (Lớp 1) khớp GT: 8822
   - Acceptor (Lớp 2) khớp AG: 8666
✅ Đã lưu: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data\maxent_1-1-1.csv

🚀 Đang xử lý Ratio: 2-1-1


Extracting: 100%|██████████| 35132/35132 [00:07<00:00, 4553.23it/s]


📊 Thống kê 2-1-1:
   - Donor (Lớp 1) khớp GT: 8822
   - Acceptor (Lớp 2) khớp AG: 8666
✅ Đã lưu: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data\maxent_2-1-1.csv

🚀 Đang xử lý Ratio: 4-1-1


Extracting: 100%|██████████| 52776/52776 [00:10<00:00, 5146.89it/s] 


📊 Thống kê 4-1-1:
   - Donor (Lớp 1) khớp GT: 8822
   - Acceptor (Lớp 2) khớp AG: 8666
✅ Đã lưu: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data\maxent_4-1-1.csv

🚀 Đang xử lý Ratio: 10-1-1


Extracting: 100%|██████████| 105708/105708 [00:20<00:00, 5145.57it/s]


📊 Thống kê 10-1-1:
   - Donor (Lớp 1) khớp GT: 8822
   - Acceptor (Lớp 2) khớp AG: 8666
✅ Đã lưu: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data\maxent_10-1-1.csv


In [10]:
# ============================================================
# --- XỬ LÝ FILE BỔ SUNG: TÙY CHỈNH ĐƯỜNG DẪN & TÊN FILE ---
# ============================================================
import os
import pandas as pd
import tqdm
from tqdm.auto import tqdm
import re

# 1. CẤU HÌNH ĐƯỜNG DẪN (Bạn thay đổi ở đây)
EXTRA_FILE_PATH = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\test_data.csv" # Đường dẫn file bạn bị sót
CUSTOM_OUTPUT_ROOT = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data" # Thư mục lưu
OUTPUT_FILE_NAME = "maxent_input_100-1-1.csv" # Tên file output bạn yêu cầu

# 2. TẠO THƯ MỤC ĐÍCH (Nếu chưa tồn tại)
os.makedirs(CUSTOM_OUTPUT_ROOT, exist_ok=True)

if not os.path.exists(EXTRA_FILE_PATH):
    print(f"⚠️ Cảnh báo: Không tìm thấy file nguồn tại: {EXTRA_FILE_PATH}")
else:
    print(f"🚀 Đang bắt đầu xử lý file: {os.path.basename(EXTRA_FILE_PATH)}")
    
    # Đọc dữ liệu
    df_extra = pd.read_csv(EXTRA_FILE_PATH)
    
    # Đồng nhất tên cột nhãn (nếu cột cũ là 'Splicing_types')
    if 'Splicing_types' in df_extra.columns:
        df_extra = df_extra.rename(columns={'Splicing_types': 'label'})
    
    # Sử dụng hàm extract_smart_motif đã định nghĩa ở các cell trước
    tqdm.pandas(desc="Processing 100-1-1")
    df_extra[['seq_5', 'seq_3']] = df_extra.progress_apply(extract_smart_motif, axis=1)
    
    # 3. KIỂM TRA & THỐNG KÊ
    d_ok = df_extra[(df_extra['label'] == 1) & (df_extra['seq_5'].str[3:5] == 'GT')].shape[0]
    a_ok = df_extra[(df_extra['label'] == 2) & (df_extra['seq_3'].str[18:20] == 'AG')].shape[0]
    
    print(f"\n📊 Thống kê kết quả:")
    print(f"   - Donor (Lớp 1) khớp chuẩn GT: {d_ok}/{len(df_extra[df_extra['label']==1])}")
    print(f"   - Acceptor (Lớp 2) khớp chuẩn AG: {a_ok}/{len(df_extra[df_extra['label']==2])}")

    # 4. LƯU FILE
    save_path = os.path.join(CUSTOM_OUTPUT_ROOT, OUTPUT_FILE_NAME)
    df_extra.to_csv(save_path, index=False)
    
    print(f"✅ HOÀN TẤT! File đã được lưu tại: {save_path}")

🚀 Đang bắt đầu xử lý file: test_data.csv


Processing 100-1-1: 100%|██████████| 938297/938297 [02:38<00:00, 5908.77it/s] 



📊 Thống kê kết quả:
   - Donor (Lớp 1) khớp chuẩn GT: 8822/8822
   - Acceptor (Lớp 2) khớp chuẩn AG: 8666/8666
✅ HOÀN TẤT! File đã được lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data\maxent_input_100-1-1.csv
